In [99]:
import pandas as pd

employee_survey = pd.read_csv('data/raw/employee_survey_data.csv')
employee_survey.head(1)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0


In [100]:
manager_survey = pd.read_csv('data/raw/manager_survey_data.csv')
manager_survey.head(1)

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3


In [101]:
general_data = pd.read_csv('data/raw/general_data.csv')
general_data.head(1)


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0


In [102]:
import zipfile
import os

def get_in_out_time(DATA_DIR: str = 'data/raw') -> tuple[pd.DataFrame, pd.DataFrame]:
    join = os.path.join
    with zipfile.ZipFile(join(DATA_DIR,'in_out_time.zip'), 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)
    in_time = pd.read_csv(join(DATA_DIR,'in_time.csv'))
    out_time = pd.read_csv(join(DATA_DIR, 'out_time.csv'))

    # rename first column to 'EmployeeID'
    in_time.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)
    out_time.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)

    return in_time, out_time

in_time, out_time = get_in_out_time()
in_time.shape, out_time.shape

((4410, 262), (4410, 262))

In [108]:
def merge_times(in_t: pd.DataFrame, out_t: pd.DataFrame):
    # out_t and in_t have the same layout:
    #  - EmployeeID
    #  - 2015-01-01_x
    #  - 2015-01-02_x
    #  - ...

    # We want to merge them on EmployeeID and Date labels (labels are the same)
    columns = in_t.columns.to_list()

    in_out_times = pd.DataFrame(columns=columns)
    in_out_times['EmployeeID'] = in_t['EmployeeID']

    for c in columns[1:]:
        in_out_times[c] = (pd.to_datetime(out_t[c]) - pd.to_datetime(in_t[c])).astype('timedelta64[m]')

    # transform all the deltas to median, avg and variance
    o = pd.DataFrame(columns=['EmployeeID', 'MedianWorkingTime', 'AvgWorkingTime', 'VarianceWorkingTime'])
    o['EmployeeID'] = in_out_times['EmployeeID']
    o['MedianWorkingTime'] = in_out_times[columns[1:]].median(axis=1)
    o['AvgWorkingTime'] = in_out_times[columns[1:]].mean(axis=1)
    o['VarianceWorkingTime'] = in_out_times[columns[1:]].var(axis=1)

    return o


in_out_time = merge_times(in_time, out_time)

PROCESS_DIR = 'data/processed'
if not os.path.exists(PROCESS_DIR):
    os.makedirs(PROCESS_DIR)

in_out_time.to_csv(os.path.join(PROCESS_DIR, 'worked_time.csv'), index=False)
in_out_time.head(1)


,EmployeeID,MedianWorkingTime,AvgWorkingTime,VarianceWorkingTime
0,1,443.0,441.935345,288.623507


In [104]:
def get_processed_data(general_data: pd.DataFrame, employee_survey: pd.DataFrame, manager_survey: pd.DataFrame, in_out_time: pd.DataFrame) -> pd.DataFrame:
    # merge general_data and employee_survey
    data = pd.merge(general_data, employee_survey, on='EmployeeID', how='left')
    # merge data and manager_survey
    data = pd.merge(data, manager_survey, on='EmployeeID', how='left')
    # merge data and in_out_time
    data = pd.merge(data, in_out_time, on='EmployeeID', how='left')

    # drop EmployeeID
    data.drop('EmployeeID', axis=1, inplace=True)

    # encode Attrition column
    data['Attrition'] = data['Attrition'].map({'Yes': 1, 'No': 0})

    # encode BusinessTravel column
    data['BusinessTravel'] = data['BusinessTravel'].map({'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2})

    # encode Department column
    data['Department'] = data['Department'].map({'Sales': 0, 'Research & Development': 1, 'Human Resources': 2})

    # encode EducationField column
    data['EducationField'] = data['EducationField'].map({'Life Sciences': 0, 'Medical': 1, 'Marketing': 2, 'Technical Degree': 3, 'Human Resources': 4, 'Other': 5})

    # encode JobRole column
    data['JobRole'] = data['JobRole'].map({'Sales Executive': 0, 'Research Scientist': 1, 'Laboratory Technician': 2, 'Manufacturing Director': 3, 'Healthcare Representative': 4, 'Manager': 5, 'Sales Representative': 6, 'Research Director': 7, 'Human Resources': 8})

    # encode Over18 column
    data['Over18'] = data['Over18'].map({'Y': 1, 'N': 0})

    # encode MaritalStatus column
    data['MaritalStatus'] = data['MaritalStatus'].map({'Single': 0, 'Married': 1, 'Divorced': 2})

    # encode Gender column
    data['Gender'] = data['Gender'].map({'Male': 0, 'Female': 1})

    # Set all the NaN values to 0
    data.fillna(0, inplace=True)

    return data

In [105]:
data = get_processed_data(general_data, employee_survey, manager_survey, in_out_time)
data.head(1)

data.to_csv(os.path.join(PROCESS_DIR, 'processed_data.csv'), index=False)

In [106]:
from sklearn.model_selection import train_test_split

X = data.drop('Attrition', axis=1)
y = data['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

# We look for hyperparameters that have the most impact on the model
importances = pd.DataFrame({'feature': X_train.columns, 'importance': rf.feature_importances_})
importances.sort_values(by='importance', ascending=False, inplace=True)

importances.head(10)

,feature,importance
31,AvgWorkingTime,0.088110
30,MedianWorkingTime,0.087709
17,TotalWorkingYears,0.076895
0,Age,0.076234
11,MonthlyIncome,0.058135
19,YearsAtCompany,0.056079
21,YearsWithCurrManager,0.041886
3,DistanceFromHome,0.041096
10,MaritalStatus,0.040332
14,PercentSalaryHike,0.037796
